In [2]:
%matplotlib notebook
%reload_ext autoreload
%autoreload 2

"""
This notebook just does everything in ./sh/preprocess.sh script
in an interactive manner.
"""

import sys
import os

sys.path.append('../')
sys.path.append('../paretoviz')
from paretoviz.utils import fmt
from paretoviz.utils import vectorops as vops

from paretoviz import normalizer as nrm
from paretoviz import tradeoff_calculator as tro
from paretoviz import peeler as plr
from paretoviz import palettizer as pltz

# Do all these plots with centroid based coloring

# rawfpath = "../data/spherical/spherical-3d.out"
# rawfpath = "../data/spherical/spherical-4d.out"
# rawfpath = "../data/spherical/spherical-8d.out"

# rawfpath = "../data/knee/knee-3d.out"
# rawfpath = "../data/knee/knee-4d.out"
# rawfpath = "../data/knee/knee-8d.out"

# rawfpath = "../data/line-surface/line-3d.out"
rawfpath = "../data/line-surface/line-4d.out"
# rawfpath = "../data/line-surface/line-6d.out" # fails
# rawfpath = "../data/line-surface/line-8d.out"

# Do all these plots with constraint based coloring

# rawfpath = "../data/knee-const/knee-const-3d.out"
# rawfpath = "../data/knee-const/knee-const-4d.out"
# rawfpath = "../data/knee-const/knee-const-8d.out"

# rawfpath = "../data/knee-const-mod/knee-const-mod-3d.out"
# rawfpath = "../data/knee-const-mod/knee-const-mod-4d.out"

# rawfpath = "../data/isolated/isolated-3d.out"
# rawfpath = "../data/isolated/isolated-4d.out"
# rawfpath = "../data/isolated/isolated-8d.out"

# rawfpath = "../data/c2dtlz2/c2dtlz2-3d.out"
# rawfpath = "../data/c2dtlz2/c2dtlz2-4d.out"
# rawfpath = "../data/c2dtlz2/c2dtlz2-5d.out"
# rawfpath = "../data/c2dtlz2/c2dtlz2-8d.out"

# rawfpath = "../data/c2dtlz2/c2dtlz2-c1/c2dtlz2-c1-3d.out"
# rawfpath = "../data/c2dtlz2/c2dtlz2-c2/c2dtlz2-c2-3d.out"
# rawfpath = "../data/c2dtlz2/c2dtlz2-c3/c2dtlz2-c3-3d.out"
# rawfpath = "../data/c2dtlz2/c2dtlz2-c4/c2dtlz2-c4-3d.out"

# rawfpath = "../data/osy/osy-3d.out"
# rawfpath = "../data/osy/osy-4d.out"

# some engineering design problems

# rawfpath = "../data/carcrash/carcrash-3d.out"
# rawfpath = "../data/carcrash/carcrash-c1/carcrash-c1-3d.out"
# rawfpath = "../data/carcrash/carcrash-c2/carcrash-c2-3d.out"

# rawfpath = "../data/gaa-das/gaa-das-10d.out"
# rawfpath = "../data/gaa-lhs/gaa-lhs-10d.out"

# some ML data sets

# rawfpath = "../data/airofoil/airofoil-5d.out"
# rawfpath = "../data/banknote/banknote-4d.out"
# rawfpath = "../data/blood/blood-4d.out" # fails
# rawfpath = "../data/cccp/cccp-4d.out"
# rawfpath = "../data/concrete/concrete-8d.out" # fails
# rawfpath = "../data/iris/iris-4d.out" 
# rawfpath = "../data/mammogram/mammogram-5d.out" # fails
# rawfpath = "../data/wil/wil-7d.out" # fails
# rawfpath = "../data/yeast/yeast-8d.out" #  fails 

path, rawfname = os.path.split(rawfpath)
normfname = rawfname.split('.')[0] + "-norm.out"
mufname = normfname.split('.')[0] + "-mu.out"

In [3]:
# This part does the normalizer.py operation

vals = fmt.load(rawfpath)

print("Normalizing {0:d} data points.".format(len(vals)))
vals_ = nrm.normalize(vals)
fmt.cat(vals_)

path, rawfile = os.path.split(rawfpath)
normfile = os.path.join(path, rawfile.split('.')[0] + "-norm.out")
print("Saving normalized data into {0:s} ...".format(normfile))
fmt.save(vals_, normfile)

Normalizing 1292 data points.
0.0000e+00	0.0000e+00	0.0000e+00	1.0000e+00
4.2074e-03	4.2256e-03	4.2685e-03	9.9200e-01
8.4149e-03	8.4512e-03	8.5369e-03	9.8400e-01
1.2622e-02	1.2677e-02	1.2805e-02	9.7599e-01
1.6830e-02	1.6902e-02	1.7074e-02	9.6799e-01
2.1037e-02	2.1128e-02	2.1342e-02	9.5999e-01
2.5245e-02	2.5353e-02	2.5611e-02	9.5199e-01
2.9452e-02	2.9579e-02	2.9879e-02	9.4399e-01
3.3660e-02	3.3805e-02	3.4148e-02	9.3599e-01
3.7867e-02	3.8030e-02	3.8416e-02	9.2798e-01
4.2074e-02	4.2256e-02	4.2685e-02	9.1998e-01
4.6282e-02	4.6481e-02	4.6953e-02	9.1198e-01
5.0489e-02	5.0707e-02	5.1222e-02	9.0398e-01
5.4697e-02	5.4932e-02	5.5490e-02	8.9598e-01
5.8904e-02	5.9158e-02	5.9758e-02	8.8797e-01
6.3112e-02	6.3384e-02	6.4027e-02	8.7997e-01
6.7319e-02	6.7609e-02	6.8295e-02	8.7197e-01
7.1526e-02	7.1835e-02	7.2564e-02	8.6397e-01
7.5734e-02	7.6060e-02	7.6832e-02	8.5597e-01
7.9941e-02	8.0286e-02	8.1101e-02	8.4797e-01
8.4149e-02	8.4512e-02	8.5369e-02	8.3996e-01
8.8356e-02	8.8737e-02	8.9638e-02	8.3196e-01
9.

In [4]:
# This part does the tradeoff_calculator.py operation

# epsilon = 0.33 # spherical
# epsilon = 0.25 # knee
epsilon = 0.025 # line
# epsilon = 0.125 # knee-const, mod
# epsilon = 0.33 # isolated-3d, 4d
# epsilon = 0.025 # isolated-8d
# epsilon = 0.33 # c2dtlz2-3d, 4d
# epsilon = 0.15 # c2dtlz2-5d
# epsilon = 0.25 # carcrash
# epsilon = 0.33 # osy

normfpath = os.path.join(path, normfname)
if os.path.exists(normfpath):
    points = fmt.load(normfpath)

    mu = tro.compute_tradeoff(points, epsilon, normalize = False)
    fmt.cat(mu)

    path, normfname = os.path.split(normfpath)
    mufpath = os.path.join(path, normfname.split('.')[0] + "-mu.out")
    print("Saving tradeoff values to {0:s} ...".format(mufpath))
    fmt.save(mu, mufpath)
else:
    print("Error: " + normfpath + " does not exist.")

Computing neighborhood ...
Computing tradeoff values with epsilon = 0.0250...
1.5870e+00
6.2985e-01
6.2985e-01
6.3025e-01
6.2982e-01
6.2985e-01
6.2982e-01
6.2985e-01
6.2977e-01
6.3024e-01
6.2982e-01
6.2985e-01
6.2982e-01
6.2985e-01
6.3027e-01
6.2977e-01
6.2985e-01
6.2982e-01
6.2985e-01
6.2982e-01
6.3022e-01
6.2987e-01
6.2982e-01
6.2985e-01
6.3000e-01
6.3017e-01
6.3031e-01
6.2943e-01
6.2943e-01
6.2967e-01
6.3056e-01
6.2992e-01
6.2943e-01
6.2967e-01
6.2992e-01
6.2943e-01
6.3031e-01
6.2943e-01
6.2992e-01
6.2943e-01
6.2967e-01
6.3031e-01
6.2992e-01
6.2992e-01
6.2943e-01
6.2992e-01
6.2943e-01
6.3007e-01
6.2943e-01
6.3017e-01
6.2943e-01
6.2967e-01
6.3007e-01
6.2992e-01
6.2992e-01
6.2943e-01
6.2992e-01
6.2992e-01
6.3031e-01
6.2943e-01
6.2967e-01
6.2992e-01
6.2992e-01
6.3031e-01
6.2943e-01
6.2992e-01
6.2943e-01
6.2967e-01
6.3007e-01
6.3056e-01
6.2943e-01
6.2943e-01
6.2967e-01
6.2992e-01
6.3031e-01
6.2992e-01
6.2992e-01
6.2992e-01
6.2992e-01
6.3007e-01
6.2943e-01
6.2992e-01
6.2992e-01
6.2992e-0

In [5]:
# This part does the peeler.py operation

# mode = "default"
mode = "no-project" # use it for line-surface

normfpath = os.path.join(path, normfname)
points = fmt.load(normfpath)

m = len(points[0])
print("Peeling data point cloud in {0:s} mode ...".format(mode))
if mode == "default":
    # the defualt mode will project the points 
    # on a simplex on the first quadrant
    ppoints = plr.project(points)
    cpoints = plr.collapse(ppoints, dim = m - 1)
    boundaries = plr.peel(cpoints)
elif mode == "no-project":
    boundaries = plr.peel(points)
fmt.cat(boundaries, dtype = 'int')

layerfpath = os.path.join(path, normfname.split('.')[0] + "-layers.out")
print("Saving {0:d} layers into {1:s} ...".format(len(boundaries), layerfpath))
fmt.save(boundaries, layerfpath, dtype = 'int')

Peeling data point cloud in no-project mode ...
Total points: 1292
Computing alpha-shape ...
alpha-shape done.
Layer 0: points added 116, points left 1176.
Computing alpha-shape ...
alpha-shape done.
Layer 1: points added 148, points left 1028.
Computing alpha-shape ...
alpha-shape done.
Layer 2: points added 155, points left 873.
Computing alpha-shape ...
alpha-shape done.
Layer 3: points added 173, points left 700.
Computing alpha-shape ...
alpha-shape done.
Layer 4: points added 153, points left 547.
Computing alpha-shape ...
alpha-shape done.
Layer 5: points added 146, points left 401.
Computing alpha-shape ...
alpha-shape done.
Layer 6: points added 126, points left 275.
Computing alpha-shape ...
alpha-shape done.
Layer 7: points added 100, points left 175.
Computing alpha-shape ...
alpha-shape done.
Layer 8: points added 85, points left 90.
Computing alpha-shape ...
alpha-shape done.
Layer 9: points added 60, points left 30.
Computing alpha-shape ...
alpha-shape done.
Layer 10: p

In [6]:
# This part does the palettizer.py operation

n_layers = 4
# mode = "invsc"
mode = "invrv"
# mode = "rv"
# mode = "sc"

normfpath = os.path.join(path, normfname)
points = fmt.load(normfpath)

layerfile = normfname.split('.')[0] + "-layers.out"
layers = fmt.load(os.path.join(path, layerfile), dtype = 'int')

print("Computing palette coordinates with {0:d} layers in {1:s} mode ...".format(n_layers, mode))
if mode == "invsc":
    palette_coords = pltz.palettize_sc(points, layers, n_layers = n_layers, inverted = True)
    palettefpath = os.path.join(path, normfname.split('.')[0] + "-palette-invsc.out")
elif mode == "invrv":
    palette_coords = pltz.palettize_rv(points, layers, n_layers = n_layers, inverted = True)
    palettefpath = os.path.join(path, normfname.split('.')[0] + "-palette-invrv.out")
elif mode == "rv":
    palette_coords = pltz.palettize_rv(points, layers, n_layers = n_layers)
    palettefpath = os.path.join(path, normfname.split('.')[0] + "-palette-rv.out")
elif mode == "sc":
    palette_coords = pltz.palettize_sc(points, layers, n_layers = n_layers)
    palettefpath = os.path.join(path, normfname.split('.')[0] + "-palette-sc.out")
else:
    print("Error: unknown mode \'{0:s}\'\n".format(mode))
    sys.exit(1)

print("Saving palette coordinates into {0:s} ...".format(palettefpath))
pltz.save_palette(palette_coords, palettefpath)

Computing palette coordinates with 4 layers in invrv mode ...
Saving palette coordinates into ../data/line-surface/line-4d-norm-palette-invrv.out ...
